In [1]:
# 问题：
#    1.分位数计算 --- 线性方法 （不同的方法会带来不同的结果）  但R语言应该采用的是linear方法 
#    2.分位数的值 --- 值的变化会对结果带来明显影响  ---  有一定可能R语言里的分位数和python中的分位数存在些许差异   但感觉可能性不大
#    3.R语言里的gev和lmoment  ---  感觉在python中的表示存在着一定的问题  想要着重考虑
# 解决方案：
#    1/2 --- 对应分位数选择和选取的方法
#    3 --- 对应lmoments3库

import pandas as pd
from scipy.stats import genextreme
from scipy import interpolate
import numpy as np
import lmoments3 as lm3
import lmoments as lm

def find_mf_func(stations_tab, amax_tab, start_year):
    megafloods_df = pd.DataFrame(columns=['id', 'LAT', 'LON', 'area', 'Q_MF', 'year_MF', 'date_MF'])
  
    nn = 0  # count number of outliers
    for i in range(len(stations_tab)):
        station_id = stations_tab['id'][i]
        dummy = amax_tab[amax_tab['id'] == station_id].dropna(subset=['q'])
        # dummy = amax_tab[amax_tab['id'] == station_id].dropna(subset=['q'])
        # dummy = amax_tab[amax_tab['id'] == station_id]
        
        if len(dummy) < 10:
            continue  # exclude short series
        
        # megafloods criteria:
        
        # dummy_quant = dummy['q'].quantile([0.25, 0.75])
        # dummy_upper = dummy_quant.loc[0.75] + 3 * (dummy_quant.loc[0.75] - dummy_quant.loc[0.25])
        # index_outlier = dummy[dummy['q'] >= dummy_upper].index

        # 1. outlier 
        dummy_sorted = sorted(dummy["q"])
        n = len(dummy_sorted)
        q1_index = int(0.25 * n) 
        q3_index = int(0.75 * n) 
        q1 = dummy_sorted[q1_index]
        q3 = dummy_sorted[q3_index]
        dummy_upper = q3 + 3 * (q3 - q1)
        # # 分位数计算，很怪  还行

        index_outlier = dummy[dummy['q'] >= dummy_upper].index
        if len(index_outlier) == 0:
            continue
        
        index_MF = []
        for tt in index_outlier:
            ty = dummy['year'][tt]
            # nyears_before = len(dummy[dummy['year'] < ty])   # 可能存在的问题 在下一行改了之后并没有明显影响
            nyears_before = len(dummy[dummy['year'] < ty]['q'])
            
            # 3. after start.year and with at least 20 years of data before event
            if nyears_before < 20 or ty < start_year:
                continue
            
        # # index_MF = []  # 存储异常事件的索引

        # # for tt in range(1, len(dummy)):
        #     qmax_old = dummy[dummy['year'] <= ty] ['q'] # 根据年份选择流量值的子集

        # # # 计算L-moments
        # # lmoments = lm3.lmom_ratios(qmax_old.values)

        # # 使用L-moments进行GEV分布拟合
        #     params = lm.pelgev(qmax_old)
        

        # # 使用pextRemes计算概率
        #     neP = genextreme.cdf(qmax_old,*params)
        #     RP = 1 / (1 - neP)
        #     RP[np.isinf(RP)] = 10**5
        #     rRP = RP[-1] / max(RP[:-1])

        #     if rRP > 3:
        #         index_MF.append(tt)

        # index_outlier = index_MF[1:]

            # 2. surprising (Tl/Tsl > 3) record-breaking event
            qmax_old = dummy[dummy['year'] <= ty]['q']    # 同上的问题
            # 计算概率分布函数
           

            local_gev = genextreme.fit(qmax_old)
            neP = 1-genextreme.pdf(qmax_old, *local_gev)   # 方法计算对吗  这是真的大问题
           
            RP = 1 / (1 - neP)
            RP[RP == float('inf')] = 10 ** 5
            rRP = RP[-1] / max(RP[:-1])
            if rRP > 3:
                index_MF.append(tt)
                
        index_outlier = index_MF
        if len(index_MF) > 0:
            for oo in index_MF:
                nn += 1
                try:
                    megafloods_df.loc[nn] = [stations_tab['id'][i], stations_tab['lat'][i], stations_tab['lon'][i],
                                         float(stations_tab['area'][i]), dummy['q'][oo], dummy['year'][oo],
                                         pd.to_datetime(
                                             f"{int(dummy['year'][oo])}-{int(dummy['month'][oo])}-{int(dummy['day'][oo])}")]
                except ValueError:
                    megafloods_df.loc[nn] = [stations_tab['id'][i], stations_tab['lat'][i], stations_tab['lon'][i],
                                         float(stations_tab['area'][i]), dummy['q'][oo], dummy['year'][oo],-1]
    
    return megafloods_df

stations = pd.read_csv("./gauges.csv")
flood_data = pd.read_csv("./floods.csv")
megafloods_df = find_mf_func(stations,flood_data,2000)
# print(find_mf_func(stations,flood_data,2000))

In [ ]:
import numpy as np
from scipy.stats import genextreme

# 2. surprising (Tl/Tsl>3) record-breaking event
qmax_old = dummy['q'][dummy['year'] <= ty].tolist()

# 进行GEV拟合
c, loc, scale = genextreme.fit(qmax_old)

# 计算概率分布函数
neP = genextreme.sf(qmax_old, c, loc=loc, scale=scale)

# 计算RP和rRP
RP = 1 / (1 - neP)
RP[np.isinf(RP)] = 10 ** 5
rRP = RP[-1] / max(RP[:-1])

if rRP > 3:
    index_MF.append(tt)


In [ ]:
# 解决方案3
import numpy as np
import lmoments as lm
import lmoments3 as lm3
# 假设 dummy 是包含年份和流量数据的数据框
# 假设 ty 是阈值年份
# 在这里，你需要根据你的数据情况对 dummy 和 ty 进行适当的赋值

index_MF = []  # 存储异常事件的索引

for tt in range(1, len(dummy)):
    qmax_old = dummy.loc[dummy['year'] <= ty, 'q']  # 根据年份选择流量值的子集

    # 计算L-moments
    lmoments = lm.lmom_ratios(qmax_old.values, nmom=3)

    # 使用L-moments进行GEV分布拟合
    params = lm.pelgev(lmoments)

    # 使用pextRemes计算概率
    neP = lm3.pextRemes(params, q=qmax_old.values)
    RP = 1 / (1 - neP)
    RP[np.isinf(RP)] = 10**5
    rRP = RP[-1] / max(RP[:-1])

    if rRP > 3:
        index_MF.append(tt)

index_outlier = index_MF[1:]


In [2]:
# 第二部分
# 1.平均值和最大值
import pandas as pd

stations = pd.read_csv("./gauges.csv")
flood_data = pd.read_csv("./floods.csv")


# 计算平均值和最大值
mean_q = flood_data.iloc[:, 1].groupby(flood_data['id']).mean().round(5)
max_q = flood_data.iloc[:, 1].groupby(flood_data['id']).max()

# 将结果赋给 stations 数据框
stations['mean_q'] = stations['id'].map(mean_q)
stations['max_q'] = stations['id'].map(max_q)

# print(stations)

In [3]:
# 第二部分 try1 
# 看上去和下面的差不多啊，问题在哪里呢？（统计库的问题）
import pandas as pd
import numpy as np  
from scipy import stats  
  
# print(stations)
def env_func(stations_tab, amax_tab):  
    # 初始化数据框  
    ENV_tab = pd.DataFrame(columns=['slope_qr_median', 'intercept_qr_median', 'slope_qr_env', 'intercept_qr_env', 'intercept_env', 'n_stat_years'])
      
    # 循环处理数据  
    for rr in range(5):  
        # 提取特定区域的数据  
        dummy_stations = stations_tab[stations_tab['regions1'] == rr]  
        dummy_q = amax_tab[amax_tab['id'].isin(dummy_stations['id'])]  
        dummy_q = dummy_q[dummy_q['q'] > 0]  
          
        # 计算log转换  
        dummy_q['qlog10'] = np.log10(dummy_q['q'])  
        dummy_q['Alog10'] = np.nan  
        for i in range(len(dummy_stations)):  
            ind_stat = np.where(dummy_q['id'] == dummy_stations['id'][i])[0][0]  
            dummy_q['Alog10'][ind_stat] = np.log10(dummy_stations['area'][i])  
          
        # 进行分位数回归  
        # Quantile regression z=0.5:  
        dummy_rq = stats.mstats.rq(formula='qlog10 ~ Alog10', data=dummy_q, tau=0.5, method='bct')  
        ENV_tab.loc[rr, 'slope_qr_median'] = dummy_rq.params[1]  
        ENV_tab.loc[rr, 'intercept_qr_median'] = dummy_rq.params[0]  
        # Quantile regression z=0.999:  
        dummy_rq = stats.mstats.rq(formula='qlog10 ~ Alog10', data=dummy_q, tau=0.999, method='bct')  
        ENV_tab.loc[rr, 'slope_qr_env'] = dummy_rq.params[1]  
        ENV_tab.loc[rr, 'intercept_qr_env'] = dummy_rq.params[0]  
        ENV_tab.loc[rr, 'intercept_env'] = np.max(dummy_q['Alog10'])  
        ENV_tab.loc[rr, 'n_stat_years'] = len(dummy_q)  
    return ENV_tab

# ENV_reg = env_func(stations, flood_data)
stations = pd.read_csv("./gauges.csv")
flood_data = pd.read_csv("./floods.csv")
print(env_func(stations, flood_data))


AttributeError: module 'scipy.stats.mstats' has no attribute 'rq'

In [1]:
import statsmodels.formula.api as smf
dir(smf)

['__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'conditional_logit',
 'conditional_mnlogit',
 'conditional_poisson',
 'gee',
 'glm',
 'glmgam',
 'gls',
 'glsar',
 'logit',
 'mixedlm',
 'mnlogit',
 'negativebinomial',
 'nominal_gee',
 'ols',
 'ordinal_gee',
 'phreg',
 'poisson',
 'probit',
 'quantreg',
 'rlm',
 'wls']

In [3]:
# 第二部分1：五个区域的包络曲线
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

def env_func(stations_tab, amax_tab):
    # Initialize an empty DataFrame for the results
    columns = ['slope_qr_median', 'intercept_qr_median', 'slope_qr_env', 'intercept_qr_env', 'intercept_env', 'n_stat_years']
    ENV_tab = pd.DataFrame(columns=columns, index=range(1, 6))
    
    for rr in range(1, 6):
        # print(rr)     打印这个东西干嘛?

        # Filter stations and amax data by region
        dummy_stations = stations_tab[stations_tab['regions1'] == rr]
        dummy_q = amax_tab[amax_tab['id'].isin(dummy_stations['id'])]
        dummy_q = dummy_q[dummy_q['q'] > 0]
        dummy_q['qlog10'] = np.log10(dummy_q['q'])
        dummy_q['Alog10'] = np.nan

        # Assign area log10 values
        for i in dummy_stations['id']:
            dummy_q.loc[dummy_q['id'] == i, 'Alog10'] = np.log10(dummy_stations.loc[dummy_stations['id'] == i, 'area'].values[0])

        # Quantile regression for tau=0.5
        model_median = smf.quantreg('qlog10 ~ Alog10', dummy_q).fit(q=0.5)
        ENV_tab.at[rr, 'slope_qr_median'] = model_median.params['Alog10']
        ENV_tab.at[rr, 'intercept_qr_median'] = model_median.params['Intercept']

        # Quantile regression for tau=0.999
        model_env = smf.quantreg('qlog10 ~ Alog10', dummy_q).fit(q=0.999)
        ENV_tab.at[rr, 'slope_qr_env'] = model_env.params['Alog10']
        ENV_tab.at[rr, 'intercept_qr_env'] = model_env.params['Intercept']

        # Calculate intercept of envelope
        intercept = -ENV_tab.at[rr, 'slope_qr_env'] * np.log10(dummy_stations['area']) + np.log10(dummy_stations['max_q'])
        ENV_tab.at[rr, 'intercept_env'] = intercept.max()

        # Number of station years
        ENV_tab.at[rr, 'n_stat_years'] = len(dummy_q)

    return ENV_tab
# stations = pd.read_csv("./gauges.csv")
# flood_data = pd.read_csv("./floods.csv")
# print(env_func(stations, flood_data))
ENV_reg = env_func(stations, flood_data)



In [4]:
# 第二部分2：EDT1 and EDT2
# 未订正
import pandas as pd
import numpy as np
from math import log10

# EDT 1
def ext_data_tab1_func(envelope_tab):
    ExtDataTable1 = pd.DataFrame(np.nan, index=envelope_tab.index, columns=["Slope_Envelope","Intercept_Envelope","Intercept_Envelope_1000km2","Slope_median_regression","Intercept_median_regression"])
    ExtDataTable1["Slope_Envelope"] = envelope_tab["slope_qr_env"]
    ExtDataTable1["Intercept_Envelope"] = envelope_tab["intercept_env"]
    ExtDataTable1["Intercept_Envelope_1000km2"] = 10 ** (envelope_tab["intercept_env"] + envelope_tab["slope_qr_env"] * log10(1000))
    ExtDataTable1["Slope_median_regression"] = envelope_tab["slope_qr_median"]
    ExtDataTable1["Intercept_median_regression"] = envelope_tab["intercept_qr_median"]
    ExtDataTable1 = ExtDataTable1.round(2)
    return ExtDataTable1

ExtDataTable1 = ext_data_tab1_func(ENV_reg)
print(ExtDataTable1)


  Slope_Envelope Intercept_Envelope Intercept_Envelope_1000km2  \
1      -0.074694           0.359792                   1.366811   
2      -0.382777           1.485215                   2.172128   
3      -0.566036           2.419434                   5.264131   
4      -0.270014           1.487376                   4.756991   
5      -0.337627           1.486842                   2.978257   

  Slope_median_regression Intercept_median_regression  
1               -0.091551                   -0.879714  
2               -0.217651                   -0.546294  
3               -0.284169                   -0.021829  
4               -0.199583                   -0.177523  
5               -0.146376                   -0.647347  


In [5]:
# EDT 2
ExtDataTable2 = pd.DataFrame(np.nan, index=range(1, 6), columns=["Number_of_gauges","Avg_Ratio_max_mean_Q","Number_of_targets", "Percentage_of_targets","Avg_Ratio_max_mean_Q_targets"])
for rr in range(1, 6):
    dummy_stations = stations[stations["regions1"] == rr]
    ExtDataTable2.loc[rr, "Number_of_gauges"] = dummy_stations.shape[0]
    ExtDataTable2.loc[rr, "Avg_Ratio_max_mean_Q"] = (dummy_stations["max_q"] / dummy_stations["mean_q"]).mean().round(2)
    dummy_mf = dummy_stations[dummy_stations["id"].isin(megafloods_df["id"])]
    ExtDataTable2.loc[rr, "Number_of_targets"] = dummy_mf.shape[0]
    ExtDataTable2.loc[rr, "Percentage_of_targets"] = (ExtDataTable2.loc[rr, "Number_of_targets"] * 100 / ExtDataTable2.loc[rr, "Number_of_gauges"]).round(1)
    ExtDataTable2.loc[rr, "Avg_Ratio_max_mean_Q_targets"] = (dummy_mf["max_q"] / dummy_mf["mean_q"]).mean().round(2)

del dummy_mf, dummy_stations

print(ExtDataTable2)

   Number_of_gauges  Avg_Ratio_max_mean_Q  Number_of_targets  \
1             671.0                  2.09                7.0   
2            3660.0                  3.25              262.0   
3             938.0                  3.50               21.0   
4            1240.0                  2.83               80.0   
5            1514.0                  2.36              121.0   

   Percentage_of_targets  Avg_Ratio_max_mean_Q_targets  
1                    1.0                          2.92  
2                    7.2                          5.40  
3                    2.2                          5.39  
4                    6.5                          4.03  
5                    8.0                          3.44  


In [2]:
print(stations)

NameError: name 'stations' is not defined

In [8]:
# 第三部分:try1   方法不对
import pandas as pd
import numpy as np
from scipy.stats import zscore

def mhq_cv_func(stations_df, amax_df, target_years):
    EU_MHQ_CV_list = {}
    for target_year in target_years:
        print(target_year)
        EU_MHQ_CV = pd.DataFrame(columns=["MHQ", "CV", "MHQ_100"])
        EU_MHQ_CV = pd.concat([stations_df[['id', 'lat', 'lon', 'area']], EU_MHQ_CV], axis=1)
        for index, station in EU_MHQ_CV.iterrows():
            dummy_df = amax_df[(amax_df['id'] == station['id']) & (amax_df['year'] < target_year)]
            dummy_df = dummy_df.dropna(subset=['q'])
            if len(dummy_df) < 10: 
                continue
            dummy_Q = dummy_df['q'].values
            dummy_Y = dummy_df['year'].values
            # Exclude outliers
            Q1 = np.quantile(dummy_Q, 0.25)
            Q3 = np.quantile(dummy_Q, 0.75)
            IQR = Q3 - Q1
            upper_bound = Q3 + 3 * IQR
            filtered_indices = dummy_Q <= upper_bound
            dummy_Q = dummy_Q[filtered_indices]
            dummy_Y = dummy_Y[filtered_indices]
            EU_MHQ_CV.at[index, 'MHQ'] = dummy_Q.mean()
            EU_MHQ_CV.at[index, 'CV'] = dummy_Q.std() / EU_MHQ_CV.at[index, 'MHQ']
        # Normalize MHQ to 100km2
        EU_MHQ_CV['MHQ_log'] = np.log10(EU_MHQ_CV['MHQ'])
        EU_MHQ_CV['area_log'] = np.log10(stations_df['area'])
        area_regr = np.polyfit(EU_MHQ_CV['area_log'], EU_MHQ_CV['MHQ_log'], 1)
        EU_MHQ_CV['MHQ_100'] = 10 ** (EU_MHQ_CV['MHQ_log'] - area_regr[0] * EU_MHQ_CV['area_log'] + area_regr[0] * np.log10(100))
        EU_MHQ_CV_list[target_year] = EU_MHQ_CV.drop(columns=['MHQ_log', 'area_log'])
    return EU_MHQ_CV_list

EU_MHQ_CV_list = mhq_cv_func(stations, flood_data, range(2000, 2023))
print(EU_MHQ_CV_list)


2000


TypeError: loop of ufunc does not support argument 0 of type numpy.float64 which has no callable log10 method

In [1]:
# 第三部分:try2 似乎流程正确，但总感觉差点什么
import pandas as pd
import numpy as np
import math

def mhq_cv_func(stations_df, amax_df, target_years):
    EU_MHQ_CV_list = {}
    for target_year in target_years:
        # print(target_year)
        EU_MHQ_CV = pd.DataFrame(columns=["MHQ", "CV", "MHQ_100"])
        EU_MHQ_CV = pd.concat([stations_df[['id', 'lat', 'lon', 'area']], EU_MHQ_CV], axis=1)
        for index, station in EU_MHQ_CV.iterrows():
            dummy_df = amax_df[(amax_df['id'] == station['id']) & (amax_df['year'] < target_year)]
            dummy_df = dummy_df.dropna(subset=['q'])
            print(f'Year: {target_year}, Station ID: {station["id"]}, Dummy DF Length: {len(dummy_df)}')
            if len(dummy_df) < 10: 
                continue
            dummy_Q = dummy_df['q'].values
            dummy_Y = dummy_df['year'].values
            # Exclude outliers
            Q1 = np.quantile(dummy_Q, 0.25)
            Q3 = np.quantile(dummy_Q, 0.75)
            IQR = Q3 - Q1
            upper_bound = Q3 + 3 * IQR
            filtered_indices = dummy_Q <= upper_bound
            dummy_Q = dummy_Q[filtered_indices]
            dummy_Y = dummy_Y[filtered_indices]
            EU_MHQ_CV.at[index, 'MHQ'] = np.mean(dummy_Q)
            EU_MHQ_CV.at[index, 'CV'] = np.std(dummy_Q) / EU_MHQ_CV.at[index, 'MHQ']
        # Normalize MHQ to 100km2
        # Normalize MHQ to 100km2
        EU_MHQ_CV['MHQ'] = EU_MHQ_CV['MHQ'].replace(0, np.nan)  # Replace 0 with NaN to avoid log(0) errors
        EU_MHQ_CV['MHQ_log'] = np.log10(EU_MHQ_CV['MHQ'].astype(float))  # Corrected
        stations_df = stations_df.dropna(subset=['area'])
        EU_MHQ_CV['area_log'] = np.log10(stations_df['area'].astype(float))  # Corrected
        area_regr = np.polyfit(EU_MHQ_CV['area_log'], EU_MHQ_CV['MHQ_log'], 1)
        EU_MHQ_CV['MHQ_100'] = 10 ** (EU_MHQ_CV['MHQ_log'] - area_regr[0] * EU_MHQ_CV['area_log'] + area_regr[0] * np.log10(100))
        EU_MHQ_CV_list[target_year] = EU_MHQ_CV.drop(columns=['MHQ_log', 'area_log'])
    return EU_MHQ_CV_list


EU_MHQ_CV_list = mhq_cv_func(stations, flood_data, range(2000, 2023))
print(EU_MHQ_CV_list)


NameError: name 'stations' is not defined

In [4]:
# 第二部分：try2 
# 还是数据统计计算的问题，以及矩阵大小似乎有点不太对
import pandas as pd
import numpy as np
import statsmodels.api as sm

# print(stations)
def env_func(stations_tab, amax_tab):
    ENV_tab = pd.DataFrame(columns=['slope_qr_median', 'intercept_qr_median', 'slope_qr_env', 'intercept_qr_env', 'intercept_env', 'n_stat_years'])

    for rr in range(5):
        dummy_stations = stations_tab[stations_tab['regions1'] == rr]
        dummy_q = amax_tab[amax_tab['id'].isin(dummy_stations['id']) & (amax_tab['q'] > 0)]

        dummy_q['qlog10'] = np.log10(dummy_q['q'] > 0)
        dummy_q['Alog10'] = np.nan
        for i in range(len(dummy_stations)):
            ind_stat = np.where(dummy_q['id'] == dummy_stations['id'][i])[0][0]
            dummy_q.at[ind_stat, 'Alog10'] = np.log10(dummy_stations['area'][i])

        # Quantile regression z=0.5
        model = sm.QuantReg(dummy_q['qlog10'], sm.add_constant(dummy_q['Alog10']))
        res = model.fit(q=0.5)
        ENV_tab.loc[rr, 'slope_qr_median'] = res.params['Alog10']
        ENV_tab.loc[rr, 'intercept_qr_median'] = res.params['const']

        # Quantile regression z=0.999
        model = sm.QuantReg(dummy_q['qlog10'], sm.add_constant(dummy_q['Alog10']))
        res = model.fit(q=0.999)
        ENV_tab.loc[rr, 'slope_qr_env'] = res.params['Alog10']
        ENV_tab.loc[rr, 'intercept_qr_env'] = res.params['const']

        ENV_tab.loc[rr, 'intercept_env'] = np.max(dummy_q['Alog10'])
        ENV_tab.loc[rr, 'n_stat_years'] = len(dummy_q)

    return ENV_tab

print(env_func(stations, flood_data))


ValueError: zero-size array to reduction operation maximum which has no identity

In [31]:
# 额外1：pathlib 库的路径查询？
from pathlib import Path
readme = Path("floods.csv").resolve()

print(readme) # resolve()和absolute()打印结果一致
print(f"Absolute path: {readme.absolute()}")
# Absolute path: /home/martin/some/path/README.md
print(f"File name: {readme.name}")
# File name: README.md
print(f"Path root: {readme.root}")
# Path root: /
print(f"Parent directory: {readme.parent}")
# Parent directory: /home/martin/some/path
print(f"File extension: {readme.suffix}")
# File extension: .md
print(f"Is it absolute: {readme.is_absolute()}")
# Is it absolute: True

E:\xiaotian\Rcode_rewrite\arcpy_code\floods.csv
Absolute path: E:\xiaotian\Rcode_rewrite\arcpy_code\floods.csv
File name: floods.csv
Path root: \
Parent directory: E:\xiaotian\Rcode_rewrite\arcpy_code
File extension: .csv
Is it absolute: True


In [24]:
# 额外2：连接路径
from pathlib import Path
# Operators:
etc = Path('/etc')
 
 
joined = etc / "cron.d" / "anacron"
print(joined)
print(f"Exists? - {joined.exists()}")
# Exists? - True

\etc\cron.d\anacron
Exists? - False


In [32]:
# 额外3：查询路径+连接路径的正确玩法
from pathlib import Path

input_str1 =Path( r'E:\xiaotian') # 输入路径前缀
input_str2 = '01.csv' # 输入路径后缀
joined = input_str1/input_str2 # 完整路径
print(joined)
readme = joined.resolve() # 查询完整路径对象
print(readme)


E:\xiaotian\01.csv
E:\xiaotian\01.csv
